# **일단 사용해보기**

## **1.환경준비**

### (1) 구글 드라이브

#### 1) 구글 드라이브 폴더 생성
* 새 폴더(langchain)를 생성하고
* 제공 받은 파일을 업로드

#### 2) 구글 드라이브 연결

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### (2) 라이브러리

#### 1) 필요한 라이브러리 설치

* requirements.txt 파일의 [경로 복사]를 한 후,
* 아래 경로에 붙여 넣기

In [2]:
# 경로 : /content/drive/MyDrive/langchain/requirements.txt
# 경로가 다른 경우 아래 코드의 경로 부분을 수정하세요.

!pip install -r /content/drive/MyDrive/langchain/requirements.txt

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

#### 2) 라이브러리 로딩

In [4]:
import pandas as pd
import numpy as np
import os
import openai

from langchain.document_loaders import PyMuPDFLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA  #← RetrievalQA를 가져오기
from langchain.chat_models import ChatOpenAI
from langchain.retrievers import WikipediaRetriever

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

### (3) OpenAI API Key 확인

In [11]:
def load_api_key(filepath):
    with open(filepath, 'r') as file:
        return file.readline().strip()

path = '/content/drive/MyDrive/langchain/'

# API 키 로드 및 환경변수 설정
openai.api_key = load_api_key(path + 'api_key.txt')
os.environ['OPENAI_API_KEY'] = openai.api_key

* ⚠️ 아래 코드셀은, 실행해서 key가 제대로 보이는지 확인하고 삭제하세요.

In [ ]:
print(os.environ['OPENAI_API_KEY'])

## **2.ChatGPT 사용해보기**

### (1) API 사용하기

#### 1) 채팅을 위한 함수 생성

In [20]:
def ask_chatgpt1(question):

    # api key 지정
    # client = openai.OpenAI()

    # # API를 사용하여 'gpt-3.5-turbo' 모델로부터 응답을 생성합니다.
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},  # 기본 역할 부여
            {"role": "user", "content": question},                          # 질문
        ]
    )

    return response.choices[0].message.content

#### 2) 사용하기

In [14]:
question = "세계에서 가장 높은 산은 무엇인가요?"
response = ask_chatgpt1(question)
print(response)

세계에서 가장 높은 산은 에베레스트 산입니다. 에베레스트 산은 네팔과 중국의 국경에 위치하고 있으며, 높이는 8,848미터(29,029피트)에 달합니다.


#### 😀실습
* 몇가지 질문을 던지고 답변을 받아 봅시다.

In [24]:
question = "독도는 어느나라땅?"
response = ask_chatgpt1(question)
print(response)

독도는 한국의 영토로 인정되어 있으며, 한국 정부가 관할하는 섬입니다. 일본도 독도를 일본의 영토로 주장하고 있지만, 국제사회에서는 한국의 영토로 인정되고 있습니다.


### (2) 역할 부여하기

#### 1) 채팅을 위한 함수 생성

In [15]:
def ask_chatgpt2(sys_role, question):

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": sys_role},
            {"role": "user", "content": question},
        ]
    )

    return response.choices[0].message.content

#### 2) 사용하기

In [23]:
sys_role = '당신은 아름답고 감동적인 시를 창작하는 데 영감을 주는 시적인 천재입니다. 당신의 시는 감정의 깊이, 자연의 아름다움, 인간 경험의 복잡성을 탐구하는 작품이며, 당신의 시를 읽는 이들의 마음을 움직입니다.'

question = "생성형 AI란 주제로 시를 지어줘. 운율에 맞춰서 작성해줘야 해."
response = ask_chatgpt2(sys_role, question)
print(response)

데이터의 미로 속 우리는  
이 세계의 시간 속에 갇혀  
숨쉬는 삶의 조각을 찾아  
삶이란 알 수 없는 수수께끼  

알고리즘과 함께 춤추며  
데이터 강을 따라 흘러가네  
유한한 이 세상 속에서  
우리는 생성형 AI의 속삭임을 듣네  

코드와 감성이 공존하는  
시간의 중심으로 나아가며  
인간의 흔적을 따라  
새로운 길을 찾으리  

빛과 그림자가 춤을 추며  
생성된 이 순간을 즐기네  
데이터의 미로 속 우리는  
생명의 노래를 부르리.


#### 😀실습
* 역할을 부여한 후 질문하여 답변을 받아 옵시다.
    * 역할 부여 사례 : 여행 플래너, 데이터분석가, 법률가, 건강관리사 등
* 2가지 이상 역할에 대해 실습해 봅시다.


In [ ]:
sys_role = '....'
question = "...."
response = ask_chatgpt2(sys_role, question)
print(response)

## **3.랭체인 사용해보기**

### (1) 위키피디아 데이터 준비

In [25]:
wiki_retriever = WikipediaRetriever(lang = "ko")

### (2) 모델준비

In [26]:
chat = ChatOpenAI(model = "gpt-3.5-turbo")

### (3) 연결(Chain)

In [27]:
wiki_chat = RetrievalQA.from_llm(llm = chat, retriever = wiki_retriever,
                             return_source_documents=True) # 정보를 가져온 원본 문서를 반환

### (4) 질문답변

* 아래 질문 부분을 수정해서 시도해 봅시다.

In [19]:
result = wiki_chat("지미카터 대통령의 출신 대학은?")
print(result["result"])

지미 카터 대통령은 조지아 공과대학교(Georgia Institute of Technology)에서 학사학위를 취득했습니다.
